In [38]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import mnist, boston_housing
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import RMSprop

In [19]:
batch_size = 128
num_classes = 10
epochs = 500

In [20]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [21]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')

404 train samples
102 test samples


In [62]:
def build_model():
    l_input = Input(shape=(train_data.shape[1],))
    x = Dense(64, activation='relu')(l_input)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    l_output = Dense(1)(x)
    
    model = Model(inputs=l_input, outputs=l_output)
    
    model.compile(loss='mse',
                  optimizer=RMSprop(),
                  metrics=['mae'])
    
    return model

In [63]:
k = 10
num_val_samples = len(train_data) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [train_data[:i * num_val_samples],
                            train_data[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [train_targets[:i * num_val_samples],
                            train_targets[(i+1)*num_val_samples:]],
                            axis=0)
    model = build_model()
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=10,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3
Processing fold # 4
Processing fold # 5
Processing fold # 6
Processing fold # 7
Processing fold # 8
Processing fold # 9


In [64]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [2.3053523778915403, 1.5957634210586549, 2.693313789367676, 3.1189573764801026, 2.204212999343872, 2.3122410774230957, 2.8030432224273683, 2.659065866470337, 3.0259777069091798, 2.2936546325683596]
mean all scores : 2.501158246994019


In [65]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

102/102 [==============================] - 1s 13ms/step


In [67]:
test_mae_score

2.736925012925092